In [1]:
%matplotlib qt
from __future__ import print_function, division
from bayesian_linear_regression import BayesianLinearRegression as BLR

import matplotlib.pyplot as plt
import numpy as np

In [2]:
def poly_with_noise(coeffs, noise_stdev, x):
    """
    Generate samples based on polynomial with coefficients coeffs 
    and additive gaussian noise with stdev noise_stdev
    @param coeffs: polynomial coefficientst starting at 0th order
    @param noise_stdev: standard deviation of additive, Gaussian noise
    @param x: input points. Must be shaped (n_samples, 1)
    """
    N = x.shape[0]
    assert x.shape[1] == 1, 'x must be shaped (n_samples, 1)'
    y = np.polynomial.polynomial.polyval(x, coeffs) + noise_stdev * np.random.randn(N).reshape(-1, 1)
    return y

def get_poly_feature(x, dim):
    """
    Generate feature for polynomial where the highest order term is raised to the power 'dim'
    e.g. dim = 2 --> quadratic
    """
    X = np.concatenate([(x**i).reshape(-1, 1) for i in range(dim)], axis=1)
    return X

In [3]:
# This section shows how to generate a prior given a dataset and assuming that the underlying function 
# is a polynomial
#
# The important parameter for the model is blr_prior_strength. 
# It should be > 1 and lower = faster adaptation, higher = smoother adaptation

# Set input range
x_min, x_max = -1., 1.

# Generate data to form a prior
N_prior = 50
noise_stdev = 0.5
coeffs = np.array([1., 1., 2., -1])
x = np.sort(np.random.uniform(x_min, x_max, N_prior)).reshape(-1, 1)
y = poly_with_noise(coeffs, noise_stdev, x)

# Set up BLR feature vector, here [1, x, x^2]
X = get_poly_feature(x, coeffs.size)

# Fit model to data
blr_prior_strength = 20  # lower = faster adaptation, higher = smoother adaptation
mdl = BLR()
mdl.set_prior_from_data(X, y, n=blr_prior_strength)
mdl.set_posterior_to_prior() # can use mdl.fit_posterior(X, y) to fit to new data. mdl.fit_posterior does not update the prior.

# Make predictions
mu_pred, stdev_pred = mdl.pred_posterior(X)
f, a = plt.subplots(1, 1)
a.plot(x, y, 'r.', label='Samples')
a.plot(x, mu_pred, 'b-', label='Predicted Mean')
a.fill(np.concatenate([x, x[::-1]]), 
       np.concatenate([mu_pred + stdev_pred, (mu_pred - stdev_pred)[::-1]]), alpha=0.5, fc='b', label=r'$1\sigma$')
a.legend()
a.set_title('Prior generated using provided data')
a.set_xlabel('x')
a.set_ylabel('y')
plt.show()

In [7]:
# Recursive updates.
# Change the underlying function and update params recursively.
# mdl.update_recursive(...) recursively updates the model parameters while keeping the prior strength fixed
# so that the model remains flexible even after data is added.
# To add data in batches, call model.fit_posterior(...). This does not update the prior. If you want to do
# That, call model.set_prior_to_posterior after model.fit_posterior(...)


# Generate samples from the new function
N_new = 100
noise_stdev_new = 0.1
coeffs_new = np.array([1., -0.5, 2., -1])
x_new = np.random.uniform(x_min, x_max, N_new).reshape(-1, 1)
y_new = poly_with_noise(coeffs_new, noise_stdev_new, x_new)

# Generate feature vector
X_new = get_poly_feature(x_new, coeffs.size)

# Update model recursively
x_plt = np.linspace(x_min, x_max, 100).reshape(-1, 1)
X_plt = get_poly_feature(x_plt, coeffs.size)

# Plot function after each update
f, a = plt.subplots(1, 1)

# plot prior samples
mu_pred, stdev_pred = mdl.pred_posterior(X_plt)

# plot prior model
a.plot(x_plt, mu_pred, 'b-', label='Predicted mean (prior)')
a.plot(x_plt, mu_pred + stdev_pred, 'b--', label=r'$1-\sigma$ (prior)')
a.plot(x_plt, mu_pred - stdev_pred, 'b--')

# plot recursively updated model
l_mu_pred, = a.plot(x_plt, mu_pred, 'c-', label='Predicted mean')
l_upper_pred, = a.plot(x_plt, mu_pred + stdev_pred, 'c--', label=r'$1-\sigma$')
l_lower_pred, = a.plot(x_plt, mu_pred - stdev_pred, 'c--')
a.plot(x, y, 'r.', label='Samples used to generate prior')
a.legend()
a.set_xlabel('x')
a.set_ylabel('y')
plt.pause(2)

# recursive updates
for i in range(N_new):
    a.plot(x_new[i], y_new[i], 'k.')
    mdl.update_recursive(X_new[i, :].reshape(1, -1), y_new[i,:].reshape(1, 1))
    mu_pred, stdev_pred = mdl.pred_posterior(X_plt)
    l_mu_pred.set_ydata(mu_pred)    
    l_upper_pred.set_ydata(mu_pred + stdev_pred)
    l_lower_pred.set_ydata(mu_pred - stdev_pred)
    a.set_title('Added sample number ' + str(i))
    plt.pause(0.1)

w_post, Sig_w_post, var_post = mdl.get_posterior_params()
print('Estimated coefficents and noise stdev:')
print('w: ', w_post.T)
print('stdev: ', np.sqrt(var_post))
print('Actual coefficients and noise stdev:')
print('w: ', coeffs_new.reshape(1, -1))
print('stdev: ', noise_stdev_new)
plt.show()    

Estimated coefficents and noise stdev:
w:  [[ 1.00026535 -0.52556632  1.98227326 -0.91061156]]
stdev:  [[0.13295837]]
Actual coefficients and noise stdev:
w:  [[ 1.  -0.5  2.  -1. ]]
stdev:  0.1
